In [1]:
!pip install ipython-autotime
# !pip install -U scikit-learn
%load_ext autotime

time: 2.42 ms (started: 2021-01-21 18:49:54 +00:00)


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
time: 2.69 ms (started: 2021-01-21 18:49:54 +00:00)


In [3]:
!pwd
import os
os.chdir('/content/drive/My Drive/ML/ml-project-master/src/')
!pwd

/content
/content/drive/My Drive/ML/ml-project-master/src
time: 237 ms (started: 2021-01-21 18:49:54 +00:00)


In [4]:
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split

from utility import *

time: 2.27 s (started: 2021-01-21 18:49:54 +00:00)


In [5]:
import keras.backend as K


def euclidean_distance_loss(y_true, y_pred):
    return K.sqrt(K.sum(K.square(y_pred - y_true), axis=-1))

time: 20.1 ms (started: 2021-01-21 18:49:57 +00:00)


# Import Dataset

In [6]:
def load_cup():
    ml_cup = np.delete(np.genfromtxt('ML-CUP20-TR.csv', 
                                     delimiter=','), obj=0, axis=1)
    return ml_cup[:, :-2], ml_cup[:, -2:]

def load_cup_blind():
    return np.delete(np.genfromtxt('ML-CUP20-TS.csv',
                                   delimiter=','), obj=0, axis=1)
    
def mean_euclidean_error(y_true, y_pred):
    assert y_true.shape == y_pred.shape
    return np.mean(np.linalg.norm(y_pred - y_true, axis=1))

time: 8.67 ms (started: 2021-01-21 18:49:57 +00:00)


In [7]:
X, y = load_cup()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

time: 32.6 ms (started: 2021-01-21 18:49:57 +00:00)


In [8]:
print(X.shape)
print(y.shape)

print(X_train.shape)
print(y_train.shape)

(1524, 10)
(1524, 2)
(1219, 10)
(1219, 2)
time: 4.07 ms (started: 2021-01-21 18:49:57 +00:00)


In [9]:
# data = np.genfromtxt('ML-CUP20-TR.csv', delimiter=',', dtype=np.float64)
# X = data[:, 1:-2]
# y = data[:, -2:]

# print(X.shape)
# print(y.shape)

time: 1.33 ms (started: 2021-01-21 18:49:57 +00:00)


In [10]:
# Xtrain, Xval, ytrain, yval = train_test_split(X, y, test_size=0.20, random_state=42)
# print(Xtrain.shape)
# print(ytrain.shape)
# print(Xval.shape)
# print(yval.shape)

time: 1.59 ms (started: 2021-01-21 18:49:57 +00:00)


# Grid Search - Gradient Descent

In [11]:
txt_best_grids = "best_results_svr_cup.txt"
grid_results_name = 'grid_results_svr_cup_v2.csv'

time: 1.29 ms (started: 2021-01-21 18:49:57 +00:00)


In [12]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVR
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import make_scorer

param_grid = {'estimator__C': [10, 11, 12, 13, 14, 15, 16, 18, 19, 20, 25, 30],
              'estimator__gamma': [0.01, 0.03, 0.05, 0.07, 0.08, 0.09, 0.1, 0.2,
                                   0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1, 2, 3, 5],
              'estimator__epsilon': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1], 
              'estimator__kernel': ['rbf']} # ['rbf', 'linear','poly','sigmoid']
              

svr = MultiOutputRegressor(SVR())
neg_mean_euclidean_error = make_scorer(mean_euclidean_error, greater_is_better=False)

grid = GridSearchCV(svr, param_grid, scoring=neg_mean_euclidean_error, cv= 3, 
                    verbose=1, n_jobs=-1, return_train_score=True)
grid_result = grid.fit(X_train, y_train)


#grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3, return_train_score=True, verbose=True)



Fitting 3 folds for each of 2280 candidates, totalling 6840 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    9.6s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:   35.4s
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done 796 tasks      | elapsed:  2.2min
[Parallel(n_jobs=-1)]: Done 1246 tasks      | elapsed:  3.5min
[Parallel(n_jobs=-1)]: Done 1796 tasks      | elapsed:  5.0min
[Parallel(n_jobs=-1)]: Done 2446 tasks      | elapsed:  6.8min
[Parallel(n_jobs=-1)]: Done 3196 tasks      | elapsed:  9.0min
[Parallel(n_jobs=-1)]: Done 4046 tasks      | elapsed: 11.5min
[Parallel(n_jobs=-1)]: Done 4996 tasks      | elapsed: 14.4min
[Parallel(n_jobs=-1)]: Done 6046 tasks      | elapsed: 17.6min
[Parallel(n_jobs=-1)]: Done 6840 out of 6840 | elapsed: 20.2min finished


time: 20min 11s (started: 2021-01-21 18:49:57 +00:00)


In [13]:
svr.get_params().keys()

dict_keys(['estimator__C', 'estimator__cache_size', 'estimator__coef0', 'estimator__degree', 'estimator__epsilon', 'estimator__gamma', 'estimator__kernel', 'estimator__max_iter', 'estimator__shrinking', 'estimator__tol', 'estimator__verbose', 'estimator', 'n_jobs'])

time: 4.18 ms (started: 2021-01-21 19:10:08 +00:00)


In [14]:
import json

grid_results_name_str = f'Results wrt: {grid_results_name} \n'
grid_params_str = f'Grid on: {json.dumps(grid_result.param_grid)} \n'
grid_results_str = f'Best: {grid_result.best_score_} using {grid_result.best_params_} \n'

with open(txt_best_grids, "a") as file_object:
    file_object.write(grid_results_name_str)
    file_object.write(grid_params_str)
    file_object.write(grid_results_str)
    file_object.write('\n')

time: 8.89 ms (started: 2021-01-21 19:10:08 +00:00)


In [15]:
print(grid_results_str)

Best: -3.0039219594900644 using {'estimator__C': 14, 'estimator__epsilon': 0.9, 'estimator__gamma': 0.08, 'estimator__kernel': 'rbf'} 

time: 1.14 ms (started: 2021-01-21 19:10:08 +00:00)


In [16]:
import pandas as pd

df = pd.DataFrame(grid.cv_results_)[['param_estimator__kernel', 'param_estimator__epsilon', 
                                     'param_estimator__C', 'param_estimator__gamma', 
                                     'mean_test_score', 'std_test_score', 
                                     'mean_train_score', 'std_train_score',
                                     'rank_test_score', 'mean_fit_time']].sort_values(by='rank_test_score')
df.rename(columns={'param_estimator__kernel': 'kernel',
                   'param_estimator__epsilon': 'epsilon',
                   'param_estimator__C': 'C',
                   'param_estimator__gamma': 'gamma',
                   'mean_test_score': 'mean_val_score',
                   'std_test_score':'std_val_score',
                   'rank_test_score': 'rank_val_score'}, inplace=True)
df.mean_train_score *= -1
df.mean_val_score *= -1
df

,kernel,epsilon,C,gamma,mean_val_score,std_val_score,mean_train_score,std_train_score,rank_val_score,mean_fit_time
916,rbf,0.9,14,0.08,3.003922,0.074211,2.554998,0.041625,1,0.174709
727,rbf,0.9,13,0.09,3.004074,0.082777,2.519006,0.043345,2,0.174222
726,rbf,0.9,13,0.08,3.004114,0.071871,2.570052,0.041714,3,0.168338
1106,rbf,0.9,15,0.08,3.004329,0.076670,2.540894,0.041641,4,0.184717
537,rbf,0.9,12,0.09,3.004341,0.081881,2.536055,0.043923,5,0.172062
...,...,...,...,...,...,...,...,...,...,...
2241,rbf,0.8,30,5,16.470649,0.526389,1.114120,0.000462,2276,0.280743
2070,rbf,0.9,25,5,16.475544,0.524287,1.336470,0.014431,2277,0.281774
2260,rbf,0.9,30,5,16.479594,0.525724,1.250838,0.000780,2278,0.281796
2089,rbf,1,25,5,16.484698,0.523677,1.466811,0.013278,2279,0.270161


time: 78.4 ms (started: 2021-01-21 19:10:08 +00:00)


In [17]:
df.to_csv(grid_results_name)

time: 40 ms (started: 2021-01-21 19:10:09 +00:00)


In [18]:
# means = grid_result.cv_results_['mean_test_score']
# stds = grid_result.cv_results_['std_test_score']
# params = grid_result.cv_results_['params']
# for mean, stdev, param in zip(means, stds, params):
#     print("%f (%f) with: %r" % (mean, stdev, param))

# import csv

# with open(single_grid_results_filename, mode='w') as grid:
#     grid = csv.writer(grid, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
#     for mean, stdev, param in zip(means, stds, params):
#       grid.writerow([mean, stdev, param])

time: 3.03 ms (started: 2021-01-21 19:10:09 +00:00)


In [19]:
# from sklearn.externals import joblib

# # #save your model or results
# # joblib.dump(grid_result, 'gs_object_monk3.pkl')
# # #load your model for further usage
# # boh = joblib.load("gs_object_monk3.pkl")

# # joblib.dump(grid.best_estimator_, 'gs_best_estimator_monk3.pkl', compress = 1)
# # filename = 'finalized_model.sav'
# # pickle.dump(model, open(filename, 'wb'))

time: 2 ms (started: 2021-01-21 19:10:09 +00:00)
